In [1]:
import numpy as np
import os
from pathlib import Path
import sklearn
import tensorflow as tf
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    
###넘파이 트릭으로 인해 50-10000은 반복연산되어 결국 10000의 크기를 가지게 된다
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10+10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20+20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [4]:
### 51타임스텝의 시계열 데이터 생성 앞의 50개가 X마지막 1개가 y
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 1))

In [5]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.021630751

In [28]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 171us/sample - loss: 0.0755 - val_loss: 0.0271
Epoch 2/20
7000/7000 [==============================] - 0s 62us/sample - loss: 0.0165 - val_loss: 0.0116
Epoch 3/20
7000/7000 [==============================] - 0s 63us/sample - loss: 0.0100 - val_loss: 0.0091
Epoch 4/20
7000/7000 [==============================] - 0s 58us/sample - loss: 0.0085 - val_loss: 0.0080
Epoch 5/20
7000/7000 [==============================] - 0s 57us/sample - loss: 0.0075 - val_loss: 0.0072
Epoch 6/20
7000/7000 [==============================] - 0s 60us/sample - loss: 0.0067 - val_loss: 0.0065
Epoch 7/20
7000/7000 [==============================] - 0s 59us/sample - loss: 0.0061 - val_loss: 0.0060
Epoch 8/20
7000/7000 [==============================] - 0s 57us/sample - loss: 0.0056 - val_loss: 0.0055
Epoch 9/20
7000/7000 [==============================] - 0s 66us/sample - loss: 0.0052 - val_loss: 0.0051
Epoch 

# RNN

In [31]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 6s 799us/sample - loss: 0.1538 - val_loss: 0.0921
Epoch 2/20
7000/7000 [==============================] - 5s 668us/sample - loss: 0.0544 - val_loss: 0.0272
Epoch 3/20
7000/7000 [==============================] - 4s 636us/sample - loss: 0.0173 - val_loss: 0.0124
Epoch 4/20
7000/7000 [==============================] - 4s 631us/sample - loss: 0.0118 - val_loss: 0.0114
Epoch 5/20
7000/7000 [==============================] - 5s 670us/sample - loss: 0.0114 - val_loss: 0.0113
Epoch 6/20
7000/7000 [==============================] - 5s 659us/sample - loss: 0.0113 - val_loss: 0.0112
Epoch 7/20
7000/7000 [==============================] - 5s 646us/sample - loss: 0.0112 - val_loss: 0.0111
Epoch 8/20
7000/7000 [==============================] - 4s 633us/sample - loss: 0.0111 - val_loss: 0.0111
Epoch 9/20
7000/7000 [==============================] - 4s 642us/sample - loss: 0.0111 - val_loss: 0.011

# 심층RNN 

In [33]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 14s 2ms/sample - loss: 0.0211 - val_loss: 0.0045
Epoch 2/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0038 - val_loss: 0.0038
Epoch 3/20
7000/7000 [==============================] - 12s 2ms/sample - loss: 0.0034 - val_loss: 0.0035
Epoch 4/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0033 - val_loss: 0.0035
Epoch 5/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0032 - val_loss: 0.0034
Epoch 6/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0032 - val_loss: 0.0033
Epoch 7/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0031 - val_loss: 0.0033
Epoch 8/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0030 - val_loss: 0.0031
Epoch 9/20
7000/7000 [==============================] - 13s 2ms/sample - loss: 0.0030 - val_loss: 0.0032
Epoch 1

In [7]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 10s 1ms/sample - loss: 0.0156 - val_loss: 0.0052
Epoch 2/20
2912/7000 [===========>..................] - ETA: 4s - loss: 0.0046

KeyboardInterrupt: 

### 타임스텝의 예측값을 순차적으로 더해 다음 값을 예측하는 경우

In [ ]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new

#predict한걸 X에 더해가며 계속해서 다음 값 예측
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
    X = np.concatenate([X, y_pred_one], axis=1)
    
Y_pred = X[:, n_steps:]

In [ ]:
Y_pred.shape

In [ ]:
np.mean(keras.metrics.mean_squared_error(Y_new, Y_pred))

### 일정량의 타임스텝을 한번에 예측하는 경우

In [8]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [51]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 10s 1ms/sample - loss: 0.0573 - val_loss: 0.0311
Epoch 2/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0251 - val_loss: 0.0189
Epoch 3/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0166 - val_loss: 0.0153
Epoch 4/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0132 - val_loss: 0.0124
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0121 - val_loss: 0.0118
Epoch 6/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0117 - val_loss: 0.0125
Epoch 7/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0112 - val_loss: 0.0124
Epoch 8/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0105 - val_loss: 0.0107
Epoch 9/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0102 - val_loss: 0.0103
Epoch 10/20
700

In [9]:
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[:, :,step_ahead -1] = series[:, step_ahead:step_ahead + n_steps, 0]
y_train = Y[:7000]
y_valid = Y[7000:9000]
y_test = Y[9000:]

In [10]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 50, 10))

In [89]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [13]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer = optimizer, metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 764us/sample - loss: 0.0480 - last_time_step_mse: 0.0381 - val_loss: 0.0351 - val_last_time_step_mse: 0.0232
Epoch 2/20
7000/7000 [==============================] - 1s 195us/sample - loss: 0.0286 - last_time_step_mse: 0.0154 - val_loss: 0.0248 - val_last_time_step_mse: 0.0111
Epoch 3/20
7000/7000 [==============================] - 1s 196us/sample - loss: 0.0228 - last_time_step_mse: 0.0089 - val_loss: 0.0208 - val_last_time_step_mse: 0.0072
Epoch 4/20
7000/7000 [==============================] - 1s 176us/sample - loss: 0.0193 - last_time_step_mse: 0.0058


KeyboardInterrupt: 

In [11]:
from tensorflow.keras.layers import LayerNormalization

class LNSimpleRNNCell(keras.layers.Layer):
    ##### 이 부분만  수정하여 normalization, 활성화 함수, drop out여부 조정 가능
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [100]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                     input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 17s 2ms/sample - loss: 0.1542 - last_time_step_mse: 0.1295 - val_loss: 0.0730 - val_last_time_step_mse: 0.0624
Epoch 2/20
7000/7000 [==============================] - 16s 2ms/sample - loss: 0.0630 - last_time_step_mse: 0.0533 - val_loss: 0.0574 - val_last_time_step_mse: 0.0486
Epoch 3/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0549 - last_time_step_mse: 0.0461 - val_loss: 0.0520 - val_last_time_step_mse: 0.0428
Epoch 4/20
7000/7000 [==============================] - 15s 2ms/sample - loss: 0.0498 - last_time_step_mse: 0.0403 - val_loss: 0.0471 - val_last_time_step_mse: 0.0360
Epoch 5/20
7000/7000 [==============================] - 16s 2ms/sample - loss: 0.0457 - last_time_step_mse: 0.0359 - val_loss: 0.0444 - val_last_time_step_mse: 0.0332
Epoch 6/20
7000/7000 [==============================] - 16s 2ms/sample - loss: 0.0432 - last_time_step_mse: 0.0334 - 

# LSTM 

In [104]:
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 698us/sample - loss: 0.0785 - last_time_step_mse: 0.0632 - val_loss: 0.0552 - val_last_time_step_mse: 0.0350
Epoch 2/20
7000/7000 [==============================] - 2s 297us/sample - loss: 0.0479 - last_time_step_mse: 0.0271 - val_loss: 0.0428 - val_last_time_step_mse: 0.0208
Epoch 3/20
7000/7000 [==============================] - 2s 297us/sample - loss: 0.0386 - last_time_step_mse: 0.0181 - val_loss: 0.0360 - val_last_time_step_mse: 0.0154
Epoch 4/20
7000/7000 [==============================] - 2s 297us/sample - loss: 0.0343 - last_time_step_mse: 0.0149 - val_loss: 0.0333 - val_last_time_step_mse: 0.0137
Epoch 5/20
7000/7000 [==============================] - 2s 295us/sample - loss: 0.0321 - last_time_step_mse: 0.0136 - val_loss: 0.0320 - val_last_time_step_mse: 0.0132
Epoch 6/20
7000/7000 [==============================] - 2s 298us/sample - loss: 0.0306 - last_time_step_mse: 0.0

# GRU셀 

In [15]:
model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 4s 593us/sample - loss: 0.0768 - last_time_step_mse: 0.0683 - val_loss: 0.0534 - val_last_time_step_mse: 0.0420
Epoch 2/20
7000/7000 [==============================] - 1s 190us/sample - loss: 0.0477 - last_time_step_mse: 0.0372 - val_loss: 0.0444 - val_last_time_step_mse: 0.0331
Epoch 3/20
7000/7000 [==============================] - 1s 191us/sample - loss: 0.0427 - last_time_step_mse: 0.0318 - val_loss: 0.0421 - val_last_time_step_mse: 0.0303
Epoch 4/20
7000/7000 [==============================] - 1s 191us/sample - loss: 0.0401 - last_time_step_mse: 0.0292 - val_loss: 0.0386 - val_last_time_step_mse: 0.0268
Epoch 5/20
7000/7000 [==============================] - 1s 193us/sample - loss: 0.0365 - last_time_step_mse: 0.0247 - val_loss: 0.0333 - val_last_time_step_mse: 0.0198
Epoch 6/20
7000/7000 [==============================] - 1s 192us/sample - loss: 0.0313 - last_time_step_mse: 0.0

In [16]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, y_valid[:, 3::2]))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 737us/sample - loss: 0.0693 - last_time_step_mse: 0.0612 - val_loss: 0.0453 - val_last_time_step_mse: 0.0366
Epoch 2/20
7000/7000 [==============================] - 1s 175us/sample - loss: 0.0393 - last_time_step_mse: 0.0314 - val_loss: 0.0357 - val_last_time_step_mse: 0.0269
Epoch 3/20
7000/7000 [==============================] - 1s 177us/sample - loss: 0.0320 - last_time_step_mse: 0.0231 - val_loss: 0.0291 - val_last_time_step_mse: 0.0189
Epoch 4/20
7000/7000 [==============================] - 1s 180us/sample - loss: 0.0270 - last_time_step_mse: 0.0166 - val_loss: 0.0252 - val_last_time_step_mse: 0.0144
Epoch 5/20
7000/7000 [==============================] - 1s 176us/sample - loss: 0.0249 - last_time_step_mse: 0.0142 - val_loss: 0.0238 - val_last_time_step_mse: 0.0129
Epoch 6/20
7000/7000 [==============================] - 1s 184us/sample - loss: 0.0236 - last_time_step_mse: 0.0